## Demonstration on Using MultiFlow and CommsFlow

This notebook is a demonstration on the use of MultiFlow and CommsFlow for the coordination of multiple devices via a single "agent". With the exception of the graphs, the code here is identical to `multiflow_demo.py` with the purpose of walking people through the step-by-step creation of the model. 

The model is a simple example of using `fmdtools` to create [System of Systems](https://en.wikipedia.org/wiki/System_of_systems) or an [Agent-Based](https://en.wikipedia.org/wiki/Agent-based_model) model. Depending on the representation and the goal of the model, it could be considered a [Multiagent System](https://en.wikipedia.org/wiki/Multi-agent_system) but the fundamental setup works all the same. 

When using `fmdtools` to do this, indivudal entities known as agents are programmed as functions, while the interactions between them are built using flows. Agents can represent a variety of things such as a representation of a more complicated system (such as a drone) or a software agent that helps control the others. The flows can be interactions such as communication channels or the environment that the agents exist in. For further explanation, see [this part of the documentation](https://nasa.github.io/fmdtools/docs-source/Development%20Guide.html#structuring-a-model)

The concept for this example is as follows: two `mover` agents move in both the x and y direction. A `coordinator` agent is tasked with sending the messages to tell them how to move within the environment.

```
Copyright © 2024, United States Government, as represented by the Administrator of the National Aeronautics and Space Administration. All rights reserved.

The “"Fault Model Design tools - fmdtools version 2"” software is licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

In [5]:
import inspect

### Bulding Flows
The first step is creating the flows and states that connect the model. This includes using a `LocationState` as the default starting location. This is given to the Communications channel which uses `CommsFlow` to allow for communications agent, and the locaiton `MultiFlow` to represent an "environment". Finally there is a parameter setup for the `Mover` agent, the purpose of which is elaborated on below.

In [6]:
from examples.multiflow_demo.multiflow_demo import LocationState, Communications, Location, MoveParam
print(inspect.getsource(LocationState))
print(inspect.getsource(Location))
print(inspect.getsource(Communications))
print(inspect.getsource(MoveParam))

class LocationState(State):
    x: float = 0.0
    y: float = 0.0

class Location(MultiFlow):
    container_s = LocationState

class Communications(CommsFlow):
    container_s = LocationState

class MoveParam(Parameter):
    x_up: float = 0.0
    y_up: float = 0.0



Next is the development of the  `Mover` agents. When the model is constructed, there will be two Movers. The first moves in the x direction and the second in the y. That is the purpose of the `MoverParam` and the if statement in the behavior function. When the model is initilized later, the two movers are sent parameters to turn on the x_up and y_up which determines how they move. 

Other parts necessary in the model are the initiation, which requires making a communications channel and a local copy of the location to track the state. The dynamic behavior function uses the increment (`inc`) function to move the device in the direction needed on each step. With that same update, it makes sure that it updates it's internal info from it's location to ensure that it can send out the correct information to the `coordinator` agent.

In [7]:
from multiflow_demo import Mover
print(inspect.getsource(Mover))    

ModuleNotFoundError: No module named 'multiflow_demo'

Finally, there is the creation of the `Coordinator` agent. It's main goal to allow for the transfer of information between the agents by receiving and transmiting information to the agents.

In [ ]:
from multiflow_demo import Coordinator
print(inspect.getsource(Coordinator))    

After building all the functions and flow, the entire model is put together and initialized with the correct parameters. Each flow and then function is added. Not the "p = {"x_up":1.0}" and "p = {"y_up":1.0} when creating the two movers. This refers back to the behavior function when making the `Mover` agent before, allowing for each one to behave in its own way.

In [ ]:
from multiflow_demo import ExModel
print(inspect.getsource(ExModel))    

Before running the model, we start with loading up the model and using the `FunctionArchitectureGraph` function to view the components and their relations to eachother. The `CommsFlow` and `MultiFlow` nodes are represented by octogons and hexagons, respectively. This is different than the circle nodes in regular flows. See `default_node_kwargs` in `fmdtools.analyze.graph` for the different node shapes and colors. 

In [ ]:
from fmdtools.define.architecture.function import FunctionArchitectureGraph

In [ ]:
mdl = ExModel()
mg = FunctionArchitectureGraph(mdl)
mg.set_edge_labels(title='')
fig, ax = mg.draw(figsize=(8,4))

Besides the model itself, we can use `FunctionArchitectureTypeGraph` to create a graph representation of hte model classes, showing the containment relationship between function classes and flow classes in the model.

In [ ]:
from fmdtools.define.architecture.function import FunctionArchitectureTypeGraph

In [ ]:
tg = FunctionArchitectureTypeGraph(mdl)
tg.set_edge_labels(title='')
fig, ax = tg.draw(figsize=(8,6))

Finally, we can run the model and view it's history via `fmdtools.sim.propagate`. The communication is setup such that mover_1 is supposed to go 25 and transmit that information to allow for mover_2 to do the same.

In [ ]:
mdl.flows["communications"].mover_1.s.x=25
mdl.flows["communications"].mover_1.send("mover_2")
from fmdtools.sim import propagate

result, mdlhist = propagate.nominal(mdl,track='all', desired_result={'graph': FunctionArchitectureGraph})
#fig = result.graph.draw(figsize=(8,6)) #graph can be drawn, but it's the same as before since it's nominal
mdlhist

In [ ]:
fig, ax = mdlhist.plot_line('flows.location.mover_1.s.x', 'flows.location.mover_1.s.y', 
                            'flows.communications.mover_1.s.x', 'flows.communications.mover_1.s.y') 